# 1.4 - Macrobond Data API for Python - Entity metadata - Release Calendar

This notebook is designed to act as a template and guidline in which certain elements can be manipulated to get the desired outcome.

Here we demonstrate how to leverage the metadata in Release entity to create a watchlist of series with information about next expected update date and time. **Please note that Macrobond only stores reliable release calendar from sources. Therefore, not all the Releases have calendar information. There might also be unscheduled revisions of series.**

You can find a full description of all methods and parameters used in the examples in the [documentation of the API](https://macrobond.github.io/macrobond-data-api/common/api.html).

*The examples use the common functions of Macrobond API. Full error handling is omitted for brevity*

***

## Importing packages

In [0]:
import pandas as pd
import macrobond_data_api as mda
import datetime

pd.options.display.max_colwidth = 10001

## Build a universe with their release entity

We are using below a list of time series from which we will extract the Release where they are coming from.
In the Macrobond's database, a Release is a dataset coming from a Source. Usually, time series belonging to a particular release have common characteristics such as a single frequency of release, commonality on the concept they refer to etc.

In [0]:
example_payload = [
    "hrbust0001",
    "krflof4520",
    "rsnaac0137",
    "itlama0989",
    "zabank0331",
    "mdinea0008",
    "czbank0739",
    "espric3581",
    "gblama0167",
    "jmrate0001",
    "clnaac0182",
    "idbopa0001",
    "narate0001",
    "deprod5135",
    "ltcons0207",
    "qarate0003",
    "eetour0055",
    "rurate0001",
    "thbank0405",
    "bgpric0639",
]

universe = mda.get_series(example_payload)

Let's now extract the releases where the time series above are coming from.

In [0]:
metadata_universe = pd.DataFrame(
    {
        "PrimName": [x.metadata["PrimName"] for x in universe],
        "LastModifiedTimeStamp": [
            x.metadata["LastModifiedTimeStamp"] for x in universe
        ],
        "Release": [
            x.metadata["Release"] if "Release" in x.metadata else None for x in universe
        ],
        "FullDescription": [x.metadata["FullDescription"] for x in universe],
        "LatestValue": [x.values[-1] for x in universe],
    }
)
if metadata_universe.Release.isnull().any():
    print(
        "Warning: The following series do not have a release, and are removed from your watchlist."
    )
    display(metadata_universe[metadata_universe.Release.isnull()])
    metadata_universe = metadata_universe[metadata_universe.Release.notnull()]

## Get calendar metadata from underlying Releases

We are now retrieving the calendar information from these releases.
We are making a use here of a few key metadata fields including:
_ NextReleaseEventTime: denoting when we expect the next calendar release
_ NextReleaseEventReferencePeriod: denoting what historical period the next release refers to e.g. "Q2 2024"

In [ ]:
release_info = mda.get_entities(metadata_universe["Release"].drop_duplicates())
release = pd.DataFrame(
    {
        "NextReleaseEventReferencePeriod": [
            x.metadata.get("NextReleaseEventReferencePeriod") for x in release_info
        ],
        "PrimName": [x.metadata["PrimName"] for x in release_info],
        "FullDescription": [x.metadata["FullDescription"] for x in release_info],
        "LastReleaseEventTime": [
            x.metadata.get("LastReleaseEventTime") for x in release_info
        ],
        "NextReleaseEventTime": [
            x.metadata.get("NextReleaseEventTime") for x in release_info
        ],
    }
)

if release.NextReleaseEventTime.isnull().any():
    print("The following series do not have a next release event time")
    display(release[release.NextReleaseEventTime.isnull()])
    # release = release[release.NextReleaseEventTime.notna()]

In [0]:
# map the release period
def release_period(date):
    today = datetime.date.today()
    tomorrow = today + datetime.timedelta(days=1)
    day_after_tomorrow = today + datetime.timedelta(days=2)
    next_monday = today + datetime.timedelta(days=(7 - today.weekday()))
    next_sunday = today + datetime.timedelta(days=(7 - today.weekday() + 6))

    if pd.isnull(date):
        return "Unknown"
    elif date < tomorrow:
        return "Today"
    elif tomorrow <= date < day_after_tomorrow:
        return "Tomorrow"
    elif day_after_tomorrow <= date < next_monday:
        return "This week"
    elif next_monday <= date <= next_sunday:
        return "Next week"
    else:
        return "Later"


release["Release Period"] = release["NextReleaseEventTime"].map(
    lambda x: release_period(x.date()) if not pd.isnull(x) else "Unknown"
)

## Merge Release calendar metadata with time series metadata

In the table below, the index shows the expected next update date based on the Source's release calendar. Moreover, in most cases, you can compare LastReleaseEventTime and LastModifiedTimeStamp below to check the difference between the expected release time based on Source calendar and the real time that Macrobond received the data.

Please note, LastReleaseEventTime is the expected, not the real release time from the source. LastModifiedTimeStamp can be overwritten by many reasons other than timeseries update.

In [ ]:
calender = (
    metadata_universe[
        [
            "PrimName",
            "Release",
            "LastModifiedTimeStamp",
            "FullDescription",
            "LatestValue",
        ]
    ]
    .merge(
        release,
        left_on="Release",
        right_on="PrimName",
        how="right",
        suffixes=("", "_release"),
    )
    .drop("PrimName_release", axis=1)
)

calender.set_index(["Release Period", "NextReleaseEventTime"], inplace=True, drop=True)
calender.sort_index(level=[1], ascending=[True], inplace=True)


calender.rename(columns={"FullDescription_release": "Release_Title"}, inplace=True)

calender["LastReleaseEventTime"] = calender["LastReleaseEventTime"].map(
    lambda x: x.strftime("%d/%m/%Y %H:%M")
)

NextReleaseEventTime_Mapped_index = calender.index.get_level_values(1).map(
    lambda x: x.strftime("%d/%m/%Y %H:%M") if not pd.isnull(x) else pd.NaT
)
calender.index = pd.MultiIndex.from_tuples(
    list(zip(calender.index.get_level_values(0), NextReleaseEventTime_Mapped_index))
)

calender = calender[
    [
        "NextReleaseEventReferencePeriod",
        "Release_Title",
        "PrimName",
        "FullDescription",
        "LatestValue",
        "LastModifiedTimeStamp",
        "LastReleaseEventTime",
    ]
]
calender